In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from matplotlib.pyplot import figure, show
from astropy.io import fits
import seaborn as sns

sns.set_theme(context="talk", style="whitegrid", font_scale=1.2, rc={"axes.formatter.use_mathtext": True})


In [2]:
# object IDs and redshifts

IDs_34 = [772,804,1109,1167,1961,8803,16125,22266,22812,26566,26788,27145,30578,31479,31873,49372,70173,71890,79534]
IDs_56 = [113,761,903,3012,6189,14285,24707,44699,47918,53927,54834,62309,79349,80185,81497,83030]

z_34 = [3.8531,3.6645,3.9183,3.6673,3.6418,3.208,3.3659,3.1371,3.0736,3.1336,3.8499,3.2001,3.1317,3.2072,3.2334,3.2435,3.0043,3.4183,3.2167]
z_56 = [5.788,5.546,5.7762,5.277,5.1834,5.4415,5.9383,5.0355,5.7774,5.8725,5.778,5.1747,5.1794,5.4926,5.3967,5.1236]

#function to open fits files
def open_fits(ID):
    filename = f"FITS/{ID}.fits"
    with fits.open(filename) as hdul:
        data = hdul[1].data 
        wavelength = data.wave
        flux = data.flux
        error = data.err
    return wavelength, flux, error

## (ii)

In [3]:
fig = figure(figsize=(15,10))
frame = fig.add_subplot()

#create dictionaries for easier look-up
id_to_z = dict(zip(IDs_34, z_34))
id_to_z.update(dict(zip(IDs_56, z_56)))

#one visualization
ID = 804
z_value = id_to_z.get(ID, None)
Ha = 6563 #angstrom

wave, flux, error = open_fits(ID)

frame.plot(wave, flux, label=f'z={z_value}')

frame.set_ylabel(r'Flux Density [$\mu Jy$]')
frame.set_xlabel(r'Wavelength [$\mu m$]')

#rest-frame
frame.axvline(Ha*1e-4,c='red',linestyle=':',label=r'H$\alpha$ (rest frame)',linewidth=.7)

#redshifted
frame.axvline((Ha*1e-4*z_value + Ha*1e-4),c='green',linestyle='--',label=r'H$\alpha$ (observed)',linewidth=.7) 
frame.set_title(f'Spectrum Visualization for ID={ID}')
frame.legend()

plt.savefig(fname='spectrum_visualization.pdf', format='pdf')
show()



## (iii)

In [4]:
import scipy.stats as scs
from scipy.optimize import curve_fit

# Example values
ID = 22812
z_value = id_to_z.get(ID, None)
Ha = 6563  
wave, flux, error = open_fits(ID)

# Observed 
Ha_obs = Ha*1e-4*(1+z_value)  # in µm

# sliced window
delta = 0.2  # µm
mask = (wave >= Ha_obs - delta) & (wave <= Ha_obs + delta)

wave_slice = wave[mask]
flux_slice = flux[mask]

# Plot full spectrum
fig = figure(figsize=(15,10))
frame = fig.add_subplot()

frame.plot(wave_slice, flux_slice, color='black',label='JWST observation')

frame.set_xlabel(r'Wavelength [$\mu m$]')
frame.set_ylabel(r'Flux Density [$\mu Jy$]')

#frame.axvline(Ha*1e-4, c='red', linestyle=':', label='Hα rest')
frame.axvline(Ha_obs, c='red', linestyle='--', label='Hα observed')
frame.set_title(r"Slice around H$_{\alpha , obs}$ with Gaussian fit for ID=22812")

# C accounts for the continuum
def gaussian(x, A, mu, sigma, C):
    return A * np.exp(-(x - mu)**2 / (2*sigma**2)) + C

params, cov = curve_fit(gaussian, wave_slice, flux_slice, p0=[1.2,Ha_obs,0.05,flux_slice.min()])

#fitting parameters
A_fit, mu_fit, sigma_fit, continuum = params

#plot fit and subtract the continuum
plt.plot(np.linspace(2.5,2.9,100), gaussian(np.linspace(2.5,2.9,100), A_fit, mu_fit, sigma_fit, continuum)-continuum, 'g--', label='Gaussian fit - continuum')
frame.legend()

area = A_fit * abs(sigma_fit) * np.sqrt(2*np.pi) #micro Jy * micro m
plt.savefig(fname="Gaussian-slice.pdf",format='pdf')
show()

print(fr"Area of the Gaussian, i.e. Ha flux: {area:.4f} micro Jansky * micro meters")

Area of the Gaussian, i.e. Ha flux: 0.0315 micro Jansky * micro meters


In [5]:
#example for the above ID
from astropy.cosmology import Planck18 as cosmo

#convert lineflux to cgs
F_Ha_cgs = (area * 1e-33 * 3e10) / ((Ha_obs *1e-4)**2) #erg s-1 cm-2

#luminosity distance
z = z_value
dL = cosmo.luminosity_distance(z)
dL_cm = dL.to('cm')

#H_alpha luminosity
L_H = 4*np.pi*dL_cm.value**2 * F_Ha_cgs

print(f"H_alpha luminosity for ID={ID}: {L_H:.2e} erg s^-1")

H_alpha luminosity for ID=22812: 1.14e+42 erg s^-1


## (iv) H_alpha luminosity for all objects

In [6]:
L_Ha_dict = {}
F_Ha_dict = {}

#go over all IDs
for ID, z_value in id_to_z.items():
    wave, flux, error = open_fits(ID)
    
    # observed
    Ha_obs = Ha*1e-4*(1+z_value)  # µm
    
    # choose some wavelength range around peak (±0.2 micron)
    delta = 0.2
    mask = (wave >= Ha_obs - delta) & (wave <= Ha_obs + delta)
    wave_slice = wave[mask]
    flux_slice = flux[mask]
    
    # gaussian
    def gaussian(x, A, mu, sigma, C):
        return A * np.exp(-(x - mu)**2 / (2*sigma**2)) + C
    

    params, cov = curve_fit(gaussian, wave_slice, flux_slice, p0=[flux_slice.max(), Ha_obs, 0.05, flux_slice.min()])

    A_fit, mu_fit, sigma_fit, continuum = params
    
    # area
    area = A_fit * abs(sigma_fit) * np.sqrt(2*np.pi)
    
    # convert to cgs
    lambda_cm = Ha_obs * 1e-4  # µm → cm
    F_Ha_cgs = (area * 1e-29 * 1e-4) * (3e10 / lambda_cm**2)  # erg/s/cm²
    
    # luminosity 
    dL = cosmo.luminosity_distance(z_value)
    dL_cm = dL.to('cm')
    L_Ha = 4 * np.pi * dL_cm.value**2 * F_Ha_cgs
    
    # save all values
    L_Ha_dict[ID] = L_Ha
    F_Ha_dict[ID] = F_Ha_cgs

for ID, L in L_Ha_dict.items():
    print(f"ID={ID}: L_Hα = {L:.2e} erg/s")

ID=772: L_Hα = 2.55e+42 erg/s
ID=804: L_Hα = 2.45e+42 erg/s
ID=1109: L_Hα = 1.04e+42 erg/s
ID=1167: L_Hα = 9.37e+40 erg/s
ID=1961: L_Hα = 3.16e+41 erg/s
ID=8803: L_Hα = 3.62e+41 erg/s
ID=16125: L_Hα = 5.03e+41 erg/s
ID=22266: L_Hα = 2.94e+41 erg/s
ID=22812: L_Hα = 1.14e+42 erg/s
ID=26566: L_Hα = 2.84e+42 erg/s
ID=26788: L_Hα = 4.47e+41 erg/s
ID=27145: L_Hα = 3.15e+42 erg/s
ID=30578: L_Hα = 9.54e+40 erg/s
ID=31479: L_Hα = 4.43e+41 erg/s
ID=31873: L_Hα = 2.41e+41 erg/s
ID=49372: L_Hα = 3.27e+41 erg/s
ID=70173: L_Hα = 1.97e+42 erg/s
ID=71890: L_Hα = 3.03e+42 erg/s
ID=79534: L_Hα = 6.58e+41 erg/s
ID=113: L_Hα = 8.07e+41 erg/s
ID=761: L_Hα = 1.22e+42 erg/s
ID=903: L_Hα = 9.89e+41 erg/s
ID=3012: L_Hα = 1.25e+42 erg/s
ID=6189: L_Hα = 2.59e+41 erg/s
ID=14285: L_Hα = 4.44e+41 erg/s
ID=24707: L_Hα = 1.53e+42 erg/s
ID=44699: L_Hα = 4.00e+41 erg/s
ID=47918: L_Hα = 6.34e+41 erg/s
ID=53927: L_Hα = 1.78e+42 erg/s
ID=54834: L_Hα = 5.74e+41 erg/s
ID=62309: L_Hα = 1.06e+42 erg/s
ID=79349: L_Hα = 1.30e+4

In [7]:
# star formation rates

# log M_star (solar masses per year) = log L_H_alpha - log (cH_alpha) [constant]

SFR_dict = {}

#formula from paper
def SFR_calculator(L):
    rate = L * 10**(-41.27)
    return rate

for ID, L in L_Ha_dict.items():
    SFR = SFR_calculator(L)
    SFR_dict[ID] = SFR
    
print(f"SFR for ID={22812}: {SFR_dict[22812]:.4} solar masses per year")

SFR for ID=22812: 6.103 solar masses per year


## (v) Stellar Mass Estimation

In [8]:
with fits.open('jades_goods-n_photometry.fits') as hdul:
    data = hdul[1].data
    #print(data.columns)
    #print(len(data.NIRSpec_ID))

lambda_rest = 1500
lambda_obs_dict = {}

# over IDs
for ID, z_value in id_to_z.items():
    lambda_obs = lambda_rest * (1 + z_value)  # observed wavelength in Å
    lambda_obs_dict[ID] = lambda_obs * 1e-4

# uncomment to print the corresponding wavelength
#for ID, lam in lambda_obs_dict.items():
#    print(f"ID={ID}: λ_obs = {lam:.2f} micron")

In [9]:
#manually stored which filters are relevant for each ID

filters_dict = {}

filters_dict[772] = ['F775W','F814W']
filters_dict[804] = ['F775W','F814W','F606W']
filters_dict[1109] = ['F775W','F814W']
filters_dict[1167] = ['F775W','F814W','F606W']
filters_dict[1961] = ['F775W','F814W','F606W']
filters_dict[8803] = ['F606W']
filters_dict[16125] = ['F606W']
filters_dict[22266] = ['F606W']
filters_dict[22812] = ['F606W']
filters_dict[26566] = ['F606W']
filters_dict[26788] = ['F775W','F814W']
filters_dict[27145] = ['F606W']
filters_dict[30578] = ['F606W']
filters_dict[31479] = ['F606W']
filters_dict[31873] = ['F606W']
filters_dict[49372] = ['F606W']
filters_dict[70173] = ['F606W']
filters_dict[71890] = ['F606W']
filters_dict[113] = ['F850LP','F105W','F090W','F115W']
filters_dict[761] = ['F850LP','F105W','F090W']
filters_dict[903] = ['F850LP','F105W','F090W','F115W']
filters_dict[3012] = ['F814W','F850LP','F105W','F090W']
filters_dict[6189] = ['F814W','F850LP','F105W','F090W']
filters_dict[14285] = ['F850LP','F105W','F090W']
filters_dict[24707] = ['F850LP','F105W','F115W']
filters_dict[44699] = ['F814W','F850LP','F105W','F090W']
filters_dict[47918] = ['F850LP','F105W','F090W','F115W']
filters_dict[53927] = ['F850LP','F105W','F115W']
filters_dict[54834] = ['F850LP','F105W','F090W','F115W']
filters_dict[62309] = ['F814W','F850LP','F105W','F090W']
filters_dict[79349] = ['F814W','F850LP','F105W','F090W']
filters_dict[80185] = ['F850LP','F105W','F090W']
filters_dict[81497] = ['F814W','F850LP','F105W','F090W']
filters_dict[83030] = ['F814W','F850LP','F105W','F090W']

In [10]:
flux_spectra = {}
i =0
#loop over all objects and their filters, if mroe than one is relevant the median is taken
for ID, filters in filters_dict.items():
    # row match
    mask = data['NIRSpec_ID'] == ID
    matched_rows = data[mask]
    
    flux_list = []
    
    for f in filters:
        col_name = f"{f}_KRON"
        # Extract all values for this column and take median if multiple rows
        flux_values = matched_rows[col_name]
        flux_list.append(flux_values)
    
    flux_median = np.median(flux_list)
    flux_spectra[ID] = flux_median * 1e-9

# uncomment the next lines if you want to check the resulting values of the flux

#for ID in flux_spectra:
#    print(f"ID {ID}: f = {flux_spectra[ID]:.2e} Jy")
    
    

In [11]:
# luminosity distances calculation for all IDs

dL_dict_pc = {}
for ID, z_value in id_to_z.items():
    dL = cosmo.luminosity_distance(z_value)
    dL_pc = dL.to('pc')
    dL_dict_pc[ID] = dL_pc

In [12]:
#calculate the flux at rest and use that in paper's formula to find the mass estimation

flux_spectra_rest = {}

for ID, f in flux_spectra.items():
    z = id_to_z.get(ID, None)
    
    # convert to rest frame
    f_rest = f / (1 + z)
    
    flux_spectra_rest[ID] = f_rest

M = {}

for ID, f_rest in flux_spectra_rest.items():
    m = -2.5*np.log10(f_rest) + 8.9
    absolute_M = m - 5*(np.log10(dL_dict_pc.get(ID, None).value)-1)
    M[ID] = absolute_M

In [13]:
#uncomment to check all absolute magnitudes

#for ID, absol in M.items():
#   print(f"Absolute Magnitude {ID}: {M[ID]:.3f}")

In [14]:
masses = {}

def mass_calculator(abs_m,z):
    M0 = -2.4*np.log10(1+z)+11
    logM = -0.5*(abs_m+19.5) + M0
    return 10**logM

for ID, abs_m in M.items():
    z = id_to_z.get(ID, None)
    masses[ID] = mass_calculator(abs_m,z)

In [15]:
for ID, mass in masses.items():
    print(f"Mass {ID}: {masses[ID]:.2e} Solar Mass")

Mass 772: 4.40e+09 Solar Mass
Mass 804: 1.22e+10 Solar Mass
Mass 1109: 5.73e+09 Solar Mass
Mass 1167: 2.47e+08 Solar Mass
Mass 1961: 8.17e+08 Solar Mass
Mass 8803: 4.29e+08 Solar Mass
Mass 16125: 1.89e+09 Solar Mass
Mass 22266: 1.83e+09 Solar Mass
Mass 22812: 7.72e+09 Solar Mass
Mass 26566: 1.16e+10 Solar Mass
Mass 26788: 7.19e+08 Solar Mass
Mass 27145: 7.57e+09 Solar Mass
Mass 30578: 1.02e+09 Solar Mass
Mass 31479: 3.70e+09 Solar Mass
Mass 31873: 3.27e+08 Solar Mass
Mass 49372: 4.41e+08 Solar Mass
Mass 70173: 2.74e+10 Solar Mass
Mass 71890: 5.90e+09 Solar Mass
Mass 113: 1.09e+09 Solar Mass
Mass 761: 1.09e+09 Solar Mass
Mass 903: 1.73e+09 Solar Mass
Mass 3012: 7.40e+08 Solar Mass
Mass 6189: 6.33e+08 Solar Mass
Mass 14285: 3.82e+08 Solar Mass
Mass 24707: 7.56e+08 Solar Mass
Mass 44699: 2.37e+08 Solar Mass
Mass 47918: 2.50e+08 Solar Mass
Mass 53927: 1.42e+09 Solar Mass
Mass 54834: 6.35e+08 Solar Mass
Mass 62309: 7.51e+08 Solar Mass
Mass 79349: 2.24e+09 Solar Mass
Mass 80185: 7.16e+08 Sol

## Mass Histograms

In [16]:
#histogram for masses distribution

masses_34 = np.array([masses[ID] for ID in IDs_34 if ID in masses]) #takes the mass of object ID only if it is in z 3-4 range
#z_34 = np.array([id_to_z[ID] for ID in IDs_34 if ID in masses])

plt.figure(figsize=(10,6))

plt.hist(masses_34, bins=30, edgecolor='k', alpha=0.7)

plt.xlabel(r"Stellar Mass [M$_\odot$]")
plt.ylabel("Galaxy count")
#plt.xscale('log')
plt.title("Stellar Mass Distribution for z:3-4 Galaxies")
plt.tight_layout()
plt.savefig(fname="Mass-Histogram.pdf",format='pdf')
#plt.show()

#histogram for masses distribution

masses_56 = np.array([masses[ID] for ID in IDs_56 if ID in masses]) #takes the mass of object ID only if it is in z 3-4 range
#z_34 = np.array([id_to_z[ID] for ID in IDs_34 if ID in masses])

plt.figure(figsize=(10,6))

plt.hist(masses_56, bins=30, edgecolor='k', alpha=0.7)

plt.xlabel(r"Stellar Mass[M$_\odot$]")
plt.ylabel("Galaxy count")
#plt.xscale('log')
plt.title("Stellar Mass Distribution for z:[5-6] Galaxies")
plt.tight_layout()
plt.savefig(fname="Mass-Histogram-56.pdf",format='pdf')
show()

In [17]:
masses_34 = np.array([masses[ID] for ID in IDs_34 if ID in masses]) #takes the mass of object ID only if it is in z 3-4 range
masses_56 = np.array([masses[ID] for ID in IDs_56 if ID in masses]) #takes the mass of object ID only if it is in z 5-6 range

plt.figure(figsize=(10,6))

plt.hist(masses_34, bins=15, edgecolor='k', alpha=.7, label="z:3-4") #histogram for z:3-4
plt.hist(masses_56, bins=10, edgecolor='k', alpha=1, color='red', label="z:5-6") #histogram for z:5-6


plt.xlabel(r"Stellar Mass [M$_\odot$]")
plt.ylabel("Galaxy count")
plt.title("Stellar Mass Distribution for z:3-4 and z:5-6 Galaxies")
plt.legend()
#plt.yscale('log')
plt.tight_layout()
plt.savefig(fname="Mass-Histogram-fused.pdf", format='pdf')
show()


## Stellar Mass vs SFR Plots


In [18]:
SFR_34 = np.array([SFR_dict[ID] for ID in IDs_34 if ID in masses])
SFR_56 = np.array([SFR_dict[ID] for ID in IDs_56 if ID in masses])

masses_34 = np.array([masses[ID] for ID in IDs_34 if ID in masses]) #takes the mass of object ID only if it is in z 3-4 range
masses_56 = np.array([masses[ID] for ID in IDs_56 if ID in masses]) #takes the mass of object ID only if it is in z 5-6 range

#order the arrays for clean plotting
sort_idx_34 = np.argsort(SFR_34)
SFR_34_sorted = SFR_34[sort_idx_34]
masses_34_sorted = np.log10(masses_34[sort_idx_34])

sort_idx_56 = np.argsort(SFR_56)
SFR_56_sorted = SFR_56[sort_idx_56]
masses_56_sorted = np.log10(masses_56[sort_idx_56])

#plotting
plt.figure(figsize=(10,6))

plt.scatter(masses_34_sorted, np.log10(SFR_34_sorted), alpha=.7, label="z:3-4") #plot for z:3-4
plt.scatter(masses_56_sorted, np.log10(SFR_56_sorted), alpha=1, color='red', label="z:5-6") #plot for z:5-6

#plt.xscale('log')
#plt.yscale('log')
plt.ylabel(r"log$_{10}$(Star Formation Rate [M$_\odot$ yr$^{-1}$])")
plt.xlabel("log$_{10}$(Stellar Mass [M$_\odot$])")
plt.title("Stellar Mass vs Star Formation Rate")
plt.legend()

plt.tight_layout()
plt.savefig(fname="mass_v_sfr.pdf", format='pdf')
show()

## Include Popesso et al. 2022

In [19]:
#find cosmic time of my galaxy groups

z34 = np.median(z_34)
z56 = np.median(z_56)

t_cosmic_34 = cosmo.age(z34).value  # in Gyr
t_cosmic_56 = cosmo.age(z56).value

#calculate 

In [20]:

def sf_main_sequence(M, t, a, da):
    a0, a1, a2, a3, a4 = a
    da0, da1, da2, da3, da4 = da
    
    M = np.array(M)
    
    x = M / 10**(a2 + a3*t)
    logSFR = a0 + a1*t - np.log10(1 + x**(-a4))
    
    common = x**(-a4) / (1 + x**(-a4))
    lnx = np.log(x)

    dloga0 = 1
    dloga1 = t
    dloga2 = a4 * common
    dloga3 = t * dloga2
    dloga4 = common * lnx / np.log(10)
    
    dlogSFR = np.sqrt(
        (dloga0*da0)**2 + (dloga1*da1)**2 + (dloga2*da2)**2 +
        (dloga3*da3)**2 + (dloga4*da4)**2
    )
    
    return logSFR, dlogSFR


In [21]:
# calculate SFR for a sensible mass range

mass = np.linspace(10**7.5, 10**10.5,1000)

# Popesso+2022 parameters and uncertainties
a = [2.693, -0.186, 10.85, -0.0729, 0.99]
da = [0.012, 0.009, 0.05, 0.0024, 0.01]

logSFR_34, dlogSFR_34 = sf_main_sequence(mass, t_cosmic_34, a, da)
logSFR_56, dlogSFR_56 = sf_main_sequence(mass, t_cosmic_56, a, da)


#plotting
plt.figure(figsize=(10,6))

plt.scatter(masses_34_sorted, np.log10(SFR_34_sorted), alpha=.7) #plot for z:3-4

plt.plot(np.log10(mass), logSFR_34, c='blue',linestyle=':',label='Popesso et al. 2022')

plt.fill_between(np.log10(mass), logSFR_34-dlogSFR_34, logSFR_34+dlogSFR_34, color='blue', alpha=0.3)


#plt.xscale('log')
#plt.yscale('log')
plt.ylabel(r"log$_{10}$(Star Formation Rate [M$_\odot$ yr$^{-1}$])")
plt.xlabel("log$_{10}$(Stellar Mass [M$_\odot$])")
plt.title("Stellar Mass vs Star Formation Rate z:[3-4]")
plt.legend()

plt.tight_layout()
plt.savefig(fname="mass_v_sfr_34.pdf", format='pdf')
show()

In [22]:
plt.figure(figsize=(10,6))
plt.scatter(masses_56_sorted, np.log10(SFR_56_sorted), alpha=1, color='red') #plot for z:5-6
plt.plot(np.log10(mass), logSFR_56, c='red',linestyle=':',label='Popesso et al. 2022')

plt.fill_between(np.log10(mass), logSFR_56-dlogSFR_56, logSFR_56+dlogSFR_56, color='red', alpha=0.3)

plt.ylabel(r"log$_{10}$(Star Formation Rate [M$_\odot$ yr$^{-1}$])")
plt.xlabel("log$_{10}$(Stellar Mass [M$_\odot$])")
plt.title("Stellar Mass vs Star Formation Rate z:[5-6]")
plt.legend()

plt.tight_layout()
plt.savefig(fname="mass_v_sfr_56.pdf", format='pdf')
show()